<a href="https://colab.research.google.com/github/ieg-dhr/NLP-Kurs_DMGK_Digitale-Geisteswissenschaften/blob/main/Data_Detective_NLP_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Detective: Uncovering Hidden Themes using NLP tasks

Created by Sarah Oberbichler [![ORCID](https://info.orcid.org/wp-content/uploads/2019/11/orcid_16x16.png)](https://orcid.org/0000-0002-1031-2759)


Welcome to this digital investigation lab! Today, we're facing an intriguing challenge:

**Our Dataset:**
A collection of newspaper pages, each containing various articles. Hidden within these pages is a common theme, connecting all the content.

**The Challenge:**
Reading through all these articles manually would be a time-consuming task, potentially taking hours or even days. Yet, somewhere in this sea of words lies our answer.

**Our Approach:**
Instead of traditional reading, we'll employ Natural Language Processing (NLP) techniques. These computational methods will help us analyze the text efficiently and uncover patterns that might not be immediately obvious to the human eye.

**Our Mission:**
Use basic NLP tools to identify the common thread running through these newspaper articles. We'll walk through this process step by step, uncovering insights along the way.

Ready to start our text analysis journey? Let's see what stories our data can tell us!

## Importing the Dataset to the Notebook

In order to access our course data, we clone the course GitHub repository to this notebook. Do do so, run the *git clone* cell below:

In [ ]:
!git clone https://github.com/ieg-dhr/NLP-Kurs_DMGK_Digitale-Geisteswissenschaften.git

In [ ]:
import pandas as pd

# Replace 'your_file.xlsx' with the actual path to your Excel file.
df = pd.read_excel('/content/NLP-Kurs_DMGK_Digitale-Geisteswissenschaften/exercises/Data_exercise_1.xlsx')

# Display the first few rows of the DataFrame to verify it's loaded correctly.
df=df[:300]
df.head(5)

## Cleaning the Dataset

Before we can start applying NLP methods to the dataset, we need to prepare the text in a way that the machine has to deal with less "noise". Less noise means punctuation marks and special characters are removed and all words are uniformly written in lowercase.

In [ ]:
import re
#Function to clean

def initial_clean(text):
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower()
    return text

df['cleaned'] = df['plainpagefulltext'].apply(initial_clean)
df['cleaned'][:5]

## 1. Basic NLP task: Tokenization

Tokenization is the process of breaking down text into smaller units called tokens, typically words or subwords. It's a fundamental step in natural language processing that enables machines to analyze and understand text by converting it into a format they can process more easily.

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

def tokenize(text):
  text = nltk.word_tokenize(text, language = 'german')
  return text

#continue the code
df['tokenized'] = df['cleaned'].apply(tokenize)
df['tokenized'][:5]

## Word Cloud
Let's visualize this bag of words and see if we already can detect our common theme

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Flatten the list of lists of lemmatized words into a single list
all_words = [word for sublist in df['tokenized'] for word in sublist]

# Create a string of all words
text = ' '.join(all_words)

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Display the generated image:
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

##  2. Basic NLP task: Lemmatization and Stop Words Removal

The lexicographic reduction of inflected forms of a word to a base form, that is, the determination of the base form of a lexeme and the arrangement of the lemmas, is also called lemmatization.

In [ ]:
import spacy
from nltk.corpus import stopwords

# Ensure the German language model is downloaded
!python -m spacy download de_core_news_sm

# Load the German language model
nlp = spacy.load('de_core_news_sm')

# Get German stop words and add custom ones
stop_words = set(stopwords.words('german'))
custom_stop_words = {'herr', 'frau', 'dez', 'januar', 'ge', 'nr', 'ſind', 'handeln'}
stop_words.update(custom_stop_words)

def lemmatize_and_remove_stopwords(texts):
    texts_out = []
    for sent in texts:
        # Join the tokens into a single string
        text = " ".join(sent)


        # Process the text with spaCy
        doc = nlp(text)

        # Lemmatize, lowercase, and remove stop words
        lemmatized = [token.lemma_.lower() for token in doc
                      if token.is_alpha and token.lemma_.lower() not in stop_words]

        texts_out.append(lemmatized)

    return texts_out

# Apply the function to the DataFrame
df['lemmatized'] = lemmatize_and_remove_stopwords(df['tokenized'])

# Display the first 5 rows of the lemmatized column
df['lemmatized'].head()


##Let's visualize the results

In [ ]:

from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Flatten the list of lists of lemmatized words into a single list
all_words = [word for sublist in df['lemmatized'] for word in sublist]

# Create a string of all words
text = ' '.join(all_words)

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Display the generated image:
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

Unfortunaltey, this word cloud does not yet give a meaningful insight into our dataset. Since many verbs and adjectives are very repetitive, those are also the most frequent in the data. Let's try to process the data further in order to focus on words that carry more meaning.

Can you guess what part of speech carries more meaning when it comes to the detection of common themes?

## Part of Speech Tagging
Part-of-speech tagging (POS tagging) refers to the process of assigning words and punctuation marks in a text to their corresponding parts of speech (word classes). This process takes into account both the definition of the word and its context (e.g., adjacent adjectives or nouns).

In [ ]:
def tagging(texts, allowed_postags=['NOUN']): # possible tags'NOUN', 'ADJ', 'ADV', 'VERB'
    texts_out = []
    nlp = spacy.load('de_core_news_sm')
    for sent in texts:
        sent_str = " ".join(sent)
        doc = nlp(sent_str)
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

df['tagging'] = tagging(df['lemmatized'])
df['tagging'][:5]

##Let's visualize the results

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Flatten the list of lists of lemmatized words into a single list
all_words = [word for sublist in df['tagging'] for word in sublist]

# Create a string of all words
text = ' '.join(all_words)

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Display the generated image:
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

It seems like we are still not able to detect any common theme.

Any idea how we could solve this problem?

## TF IDF Vectorizer

The problem with counting word occurrences is that some words are less frequent but carry the key informatio in a text or corpus.

For this reason, it is sometimes better to normalize the word counts by the number of times they appear in the documents. This is the general idea behind the tf-idf vectorization.

Tf stands for **term frequency**, the number of times the word appears in each document. We already did this before.

Idf stands for **inverse document frequency**, an inverse count of the number of documents a word appears in. Idf measures how significant a word is in the whole corpus.



In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer


# Convert the lists in 'tagging' column to strings
df['tagging_str'] = df['tagging'].apply(lambda x: ' '.join(x))

# Create and fit the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=50000)  # You can adjust max_features as needed
tfidf_matrix = tfidf_vectorizer.fit_transform(df['tagging_str'])

# Get feature names (words) and their TF-IDF scores
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.sum(axis=0).A1
print("Feature Names:", feature_names)
print("TF-IDF Scores:", tfidf_scores)

# Create a DataFrame with words and their TF-IDF scores
word_tfidf_df = pd.DataFrame({'word': feature_names, 'tfidf_score': tfidf_scores})

# Sort by TF-IDF score in descending order (most frequent/important words first)
word_tfidf_df = word_tfidf_df.sort_values('tfidf_score', ascending=False)

# Select top N words (you can adjust this number)
top_n = 1000
top_words = word_tfidf_df['word'].head(top_n).tolist()

# Create the 'j' column in the original DataFrame
df['vectorized'] = df['tagging'].apply(lambda x: [word for word in x if word in top_words])

df['vectorized']


##Let's visualize the results

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Flatten the list of lists of lemmatized words into a single list
all_words = [word for sublist in df['j'] for word in sublist]

# Create a string of all words
text = ' '.join(all_words)

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Display the generated image:
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

Can you tell what is the common theme in the dataset?